In [4]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 3
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empty dir
    DOWNLOAD_MNIST = True

train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=torchvision.transforms.ToTensor(), download=False)
test_loader = Data.DataLoader(dataset=test_data, batch_size=500, shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(64 * 7 * 7, 1024)
        self.dropout = nn.Dropout(keep_prob_rate)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def test(cnn):
    cnn.eval()  
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = cnn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print('Test Accuracy: {:.2f}%'.format(100 * accuracy))
    return accuracy

def train(cnn):
    cnn.train()  
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = cnn(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            if step % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, max_epoch, step+1, len(train_loader), loss.item()))
    test(cnn)

if __name__ == '__main__':
    cnn = CNN()
    train(cnn)


Epoch [1/3], Step [1/1200], Loss: 2.3062
Epoch [1/3], Step [101/1200], Loss: 0.9061
Epoch [1/3], Step [201/1200], Loss: 0.4974
Epoch [1/3], Step [301/1200], Loss: 0.3134
Epoch [1/3], Step [401/1200], Loss: 0.2524
Epoch [1/3], Step [501/1200], Loss: 0.5492
Epoch [1/3], Step [601/1200], Loss: 0.1671
Epoch [1/3], Step [701/1200], Loss: 0.0923
Epoch [1/3], Step [801/1200], Loss: 0.2100
Epoch [1/3], Step [901/1200], Loss: 0.1119
Epoch [1/3], Step [1001/1200], Loss: 0.0766
Epoch [1/3], Step [1101/1200], Loss: 0.1500
Epoch [2/3], Step [1/1200], Loss: 0.1297
Epoch [2/3], Step [101/1200], Loss: 0.0850
Epoch [2/3], Step [201/1200], Loss: 0.0333
Epoch [2/3], Step [301/1200], Loss: 0.2266
Epoch [2/3], Step [401/1200], Loss: 0.1496
Epoch [2/3], Step [501/1200], Loss: 0.3408
Epoch [2/3], Step [601/1200], Loss: 0.0452
Epoch [2/3], Step [701/1200], Loss: 0.3149
Epoch [2/3], Step [801/1200], Loss: 0.1002
Epoch [2/3], Step [901/1200], Loss: 0.0407
Epoch [2/3], Step [1001/1200], Loss: 0.0701
Epoch [2/3],